In [164]:
import pandas as pd
import re
import sys
import os

# Añadir la ruta del directorio data_utils al sys.path
# Agregar la carpeta raíz 'data_utils' al sys.path
sys.path.append(os.path.abspath('../'))


In [165]:
#Configuraciones
from data_utils.configuraciones.variables import variables_data

#Manipulacion de datos
from data_utils.manipulacion_de_datos.maniuplacion_de_datos import  procesar_descripcion,   \
                                                                    estandarizado_columnas, \
                                                                    cargar_datos,           \
                                                                    generar_descripcion,    \
                                                                    procesar_descripcion,   \
                                                                    limpiar_datos
from data_utils.manipulacion_de_datos.gestion_de_datos import indexar_productos

#Funciones de Busquedas
from data_utils.funciones_de_busqueda.busqueda_por_filtro import filtrar_por_laboratorio
from data_utils.funciones_de_busqueda.busquedas_por_texto import main_principal

#Aplicaciones de Inteligencia Artificial
from data_utils.utilidades_comunes.aplicaciones_de_inteligencia_artificial.procesamiento_de_lenguaje_natural.index import procesar_comparacion

Instanciamientos de Clases

In [166]:
variables_data = variables_data()


import re

def procesar_descripcion_x(data, nombre_columna):
    """
    Procesa una columna de un DataFrame aplicando normalización y manejo de combinaciones de texto.
    
    Parámetros:
    - data: DataFrame que contiene la columna a procesar.
    - nombre_columna: Nombre de la columna que se va a procesar.
    
    Retorna:
    - DataFrame con la columna procesada y actualizada.
    """
    # Verificar si la columna existe en el DataFrame
    if nombre_columna not in data.columns:
        raise KeyError(f"La columna '{nombre_columna}' no existe en el DataFrame.")
    
    # Función para normalizar el texto
    def normalizar(texto):
        # Asegurar que el texto sea un string
        texto = str(texto).strip().lower()
        
        # 1. Eliminar espacios redundantes
        texto = re.sub(r'\s+', ' ', texto)

        # 2. Remover caracteres especiales excepto letras, números, espacios y '%'
        texto = re.sub(r'[^a-zA-Z0-9\s%]', '', texto)

        # 3. Separar números de letras y letras de números
        texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
        texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)

        # 4. Manejo de 'x':
        texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)  # separar x entre números
        texto = re.sub(r'\b([a-zA-Z]+)x\b', r'\1 x', texto)  # separar palabras terminadas en x

        # 5. Reemplazar separadores como guiones o barras con espacios
        texto = re.sub(r'[-/]', ' ', texto)

        # 6. Manejo de unidades (por ejemplo, separar g, ml, etc. de los números)
        texto = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', texto)

        # 7. Eliminar espacios redundantes nuevamente
        texto = re.sub(r'\s+', ' ', texto).strip()  # <- Aquí se eliminan dobles espacios

        # 8. Combinaciones
        combinaciones = {
            'anemido x': 'anemidox',
            'jgaprell' : 'jeringa prellenada',
            'fle x': 'flex',
            'argeflo x':'argeflox',
            'aropa x': 'aropax',
            'arolte x' : 'aroltex',
            'atopi x':  'atopix',
            'atorma x': 'atormax',
            'comprec': 'comp recubierto',
            'soloft': 'solucion oftalmica',
            'g': 'gr',
            'cr': 'crema',
            'comp' : 'comprimidos',
            'caps': 'capsulas',
            "jer.pre" : "jeringa prellenada",
            "mg" : "miligramos",
            "ml" : "mililitros",
            "mm" : "milimetros",
            'zyvo x': 'zyvox',
            'loc' : 'locion',
            "gr" : "gramos",
            "jga" : 'jeringa',
            "cm" : 'centimetros',
            'u' : "unidad",
            'shamp' : 'shampoo',
            "unid" : " unidad",
            'cep' : 'cepillo dental',
            "emuls prot" : 'emulsion protectora',
            'emulshidrat':'emulsion hidratante',
            'emulshumect' : "emulsion humectante",
            'iny': 'inyeccion',
            'amp' : 'ampolla',
            'inylioffa': 'inyeccion lioffa',
            'ds' : 'dosis',
            'inyfa': 'inyeccion fa',
            'jbe': 'jarabe',
            'blist': 'blister',
            'sob' : 'sobres',
            'fa': 'frasco ampolla',
            'jab' : 'jabon',
            'cps' :'capsulas',
            'ivi x' : 'ivix',
            'inya': 'inyectable',
            'inyjgaprell' : 'iny jeringa prellenada',
            'mcg': 'microgramo',
            'gts': 'gotas',
            'mgml' : 'miligramos por mililitro',
            'env': 'envase',
            'gtsoft' : 'gotas oftálmicas',
            'compdisp': 'comprimidos disp',
            'comprecran' : 'comprimidos recubiertos ranurados',
            'sol': 'solucion',
            'jgapre': 'jeringa prellenada',
            'fcocps' : 'frasco capsulas',
            'fcogotero' : 'frasco gotero',
            'fco': 'frasco', 
            'solspray' : 'solucion spray',
            'solpuas' : 'sol puas',
            'cpsblandas' : 'cps blandas',
            'tab' : 'tableta',
            'sach' : 'sachet',
            'mgvial': 'miligramos vial',
            'cpsbl' :'capsula blanda',
            'zyvali x': 'zyvalix',
            'x' : '',
            'lapprell': 'lapicera prellenada',
            'complibprol' : 'comp lib prolongada',
            'compcu': 'comp cu',
            'jerprell'  : "jeringa prellenada",
            'prell': 'prellenada',
            'drenabprecortconv': 'drenable precorte conve',
            'drenabprecortopac': 'drenable precorte opac '        
        }
        for key, val in combinaciones.items():
            texto = re.sub(rf'\b{key}\b', val, texto)

        # 9. Eliminar cualquier doble espacio que haya quedado después de los reemplazos
        texto = re.sub(r'\s+', ' ', texto).strip()

        return texto

    # Aplicar la función 'normalizar' a cada elemento de la columna
    data[nombre_columna] = data[nombre_columna].apply(normalizar)
    
    return data

Procesamiento df Master Producto

In [167]:
#Carga de Producto
#df_Producto = cargar_datos(variables_data.csv_producto)
#df_Producto = limpiar_datos(df_Producto)
#df_Producto = indexar_productos(df_Producto,variables_data.index_productos)
#df_MasterProductos = generar_descripcion(df_Producto,variables_data.name_columna_1_MasterProductos,variables_data.name_columna_2_MasterProductos,variables_data.descripcion_limpia_MasterProductos)
#df_MasterProductos = procesar_descripcion_x(df_MasterProductos, variables_data.descripcion_limpia_MasterProductos)
#df_MasterProductos = df_MasterProductos.to_csv("df_MasterProductos.csv")




df_MasterProductos = pd.read_csv("df_MasterProductos.csv")
df_MasterProductos = procesar_descripcion_x(df_MasterProductos, variables_data.descripcion_limpia_MasterProductos)
df_MasterProductos[variables_data.descripcion_limpia_MasterProductos]

C:\Users\Windows\AppData\Local\Temp\ipykernel_2360\918448566.py:12: DtypeWarning: Columns (8,15,18,19,22,29,40,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_MasterProductos = pd.read_csv("df_MasterProductos.csv")


0                                                      nan
1                                                      nan
2        repetido supportan 500 mililitros caja 15 sach...
3        bolsas papel grado medico 175 65 420 milimetro...
4        bolsas papel grado medico 260 60 520 milimetro...
                               ...                        
56765               zyvalix 250 miligramos comprimidos 120
56766           zyvalix plus 500 miligramos comprimidos 60
56767    zyvox iv 600 miligramos bolsas 10 300 mililitr...
56768                 zyvox oral 600 miligramos tableta 10
56769                                zz top comprimidos 30
Name: descripcion_limpia_producto, Length: 56770, dtype: object

DataFrame Master Producto Filtrado por Laboratorio

In [168]:
# El usuario ingresa el nombre del laboratorio
laboratorio_input = input("Ingresa el nombre del laboratorio: ").strip()
# Filtrar productos por laboratorio
df_MasterProductos_Filtro = filtrar_por_laboratorio(df_MasterProductos, laboratorio_input)

Se encontraron 38 productos coincidentes del laboratorio 'abbvie'.


DataFrame Proveedor

In [169]:
#Carga Proveedor
cvs_proveedor = 'ABBVIE.csv'
df_Proveedor = cargar_datos(cvs_proveedor)
# Eliminar filas donde todas sus columnas están vacías
df_Proveedor = limpiar_datos(df_Proveedor)
#Indexacion 
df_Proveedor = indexar_productos(df_Proveedor,variables_data.index_proveedor)
#Normalizacion de Columnas
df_Proveedor = estandarizado_columnas(df_Proveedor,variables_data.name_columna_1_Proveedor,variables_data.name_columna_2_Proveedor,variables_data.descripcion_limpia_Proveedor)
#Normalizacion de Columna Descripcion
df_Proveedor = procesar_descripcion_x(df_Proveedor, variables_data.descripcion_limpia_Proveedor)
df_Proveedor.columns

#resultado_df = pd.DataFrame(df_Proveedor)
#resultado_df.to_csv("df_Proveedor.csv", index=False)

Index(['condición de precios', 'cod. producto', 'descripcion', 'presentacion',
       'unidades ', 'precio s/iva', 'descuento', 'total', 'index_proveedor',
       'descripcion_limpia_proveedor'],
      dtype='object')

Procesamiento CODIGO DE PRODUCTO

In [170]:
df_MasterProductos_Filtro.columns
# Guardar resultados en CSV
#resultado_df = pd.DataFrame(df_MasterProductos_Filtro)
#resultado_df.to_csv("df_MasterProductos_Filtro.csv", index=False)

Index(['Unnamed: 0', 'niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2',
       'abrev', 'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v',
       'precio', 'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v',
       'rel_v_c', 'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado',
       'cod_bonif', 'pped', 'st_min', 'st_max', 'observ', 'pc_comision',
       'pc_utilidad', 'ind_stock', 'ind_partidas', 'ind_camestado',
       'ind_series', 'ind_fvto', 'ninum', 'tipo_egreso', 'pc_diftrans',
       'cod_cvta', 'cat_item', 'cant_rep', 'pto_rep', 'atrib0', 'atrib1',
       'atrib2', 'index_producto', 'descripcion_limpia_producto'],
      dtype='object')

In [171]:
df_Proveedor

,condición de precios,cod. producto,descripcion,presentacion,unidades,precio s/iva,descuento,total,index_proveedor,descripcion_limpia_proveedor
0,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellenad...
1,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,humira ac 2 40 miligramos 2 lapiceras prellena...
2,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,2,humira ac 1 80 miligramos 1 lapicera prellenad...
3,18% DESCUENTO -ISJ,20062091,RINVOQ 15mg,15 MG x 30 COMP REC DE LIB. PROLONGADA,1.0,"2.345.085,38",18%,"1.987.360,49",3,rinvoq 15 miligramos 15 miligramos 30 comprimi...
4,NaN,20069005,RINVOQ 30mg,30 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,4,rinvoq 30 miligramos 30 miligramos 30 comprimi...
5,NaN,20078725,RINVOQ 45mg,45 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,5,rinvoq 45 miligramos 45 miligramos 30 comprimi...
6,NaN,20069017,SKYRIZI 150mg,1 LAPICERA PRELLENADA por 1 ML (150 mg),NaN,NaN,NaN,-,6,skyrizi 150 miligramos 1 lapicera prellenada p...
7,NaN,20079636,SKYRIZI 360mg,2.4ML OBI AR FP,NaN,NaN,NaN,-,7,skyrizi 360 miligramos 24 mililitros obi ar fp
8,NaN,20079635,SKYRIZI 600 mg,10.0 mL VIAL AR FP,NaN,NaN,NaN,-,8,skyrizi 600 miligramos 100 mililitros vial ar fp
9,NaN,20011756,MAVIRET,100 MG/40 MG x 84 COMP REC,NaN,NaN,NaN,-,9,maviret 100 miligramos 40 miligramos 84 compri...


Procesamiento NLP DESCRIPCION

In [172]:
df_resultado_similiridad = main_principal(df_MasterProductos_Filtro, df_Proveedor)
df_resultado_similiridad

,similaridad,condición de precios_x,cod. producto_x,descripcion_x,presentacion_x,unidades _x,precio s/iva_x,descuento_x,total_x,index_proveedor_x,...,descripcion_limpia_producto,condición de precios_y,cod. producto_y,descripcion_y,presentacion_y,unidades _y,precio s/iva_y,descuento_y,total_y,index_proveedor_y
0,0.780209,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,...,humira ac 2 20 miligramos 20 miligramos 02 mil...,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0
1,0.606657,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,...,humira ac 2 40 miligramos 40 miligramos 04 mil...,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1
2,0.944039,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,2,...,humira ac 1 80 miligramos 80 miligramos 08 mil...,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,2
3,0.861427,18% DESCUENTO -ISJ,20062091,RINVOQ 15mg,15 MG x 30 COMP REC DE LIB. PROLONGADA,1.0,"2.345.085,38",18%,"1.987.360,49",3,...,rinvoq 15 miligramos comprimidos lib prolongad...,18% DESCUENTO -ISJ,20062091,RINVOQ 15mg,15 MG x 30 COMP REC DE LIB. PROLONGADA,1.0,"2.345.085,38",18%,"1.987.360,49",3
4,0.894066,NaN,20069005,RINVOQ 30mg,30 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,4,...,rinvoq 30 miligramos comprimidos lib prolongad...,NaN,20069005,RINVOQ 30mg,30 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,4
5,0.727045,NaN,20078725,RINVOQ 45mg,45 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,5,...,rinvoq 45 miligramos comprimidos lib prolongad...,NaN,20078725,RINVOQ 45mg,45 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,5
6,0.876686,NaN,20069017,SKYRIZI 150mg,1 LAPICERA PRELLENADA por 1 ML (150 mg),NaN,NaN,NaN,-,6,...,skyrizi 150 miligramos lapicera prellenada 1,NaN,20069017,SKYRIZI 150mg,1 LAPICERA PRELLENADA por 1 ML (150 mg),NaN,NaN,NaN,-,6
7,0.352721,NaN,20079636,SKYRIZI 360mg,2.4ML OBI AR FP,NaN,NaN,NaN,-,7,...,skyrizi 360 miligramos cart 1 inyeccion,NaN,20079636,SKYRIZI 360mg,2.4ML OBI AR FP,NaN,NaN,NaN,-,7
8,0.669698,NaN,20079635,SKYRIZI 600 mg,10.0 mL VIAL AR FP,NaN,NaN,NaN,-,8,...,skyrizi 600 miligramos vial 1,NaN,20079635,SKYRIZI 600 mg,10.0 mL VIAL AR FP,NaN,NaN,NaN,-,8
9,0.587491,NaN,20011756,MAVIRET,100 MG/40 MG x 84 COMP REC,NaN,NaN,NaN,-,9,...,maviret comprimidos recubierto 84,NaN,20011756,MAVIRET,100 MG/40 MG x 84 COMP REC,NaN,NaN,NaN,-,9


Procesamiento para comparacion de  Descripciones

In [173]:
df_resultado_similiridad[df_resultado_similiridad['similaridad'] > 0.8][['similaridad', 'niprod', 'cod_prod', 'descripcion_limpia_proveedor', 'descripcion_limpia_producto', 'index_proveedor_y','codbarra']]

,similaridad,niprod,cod_prod,descripcion_limpia_proveedor,descripcion_limpia_producto,index_proveedor_y,codbarra
2,0.944039,119550.0,55604,humira ac 1 80 miligramos 1 lapicera prellenad...,humira ac 1 80 miligramos 80 miligramos 08 mil...,2,8054083017648
3,0.861427,119559.0,56945,rinvoq 15 miligramos 15 miligramos 30 comprimi...,rinvoq 15 miligramos comprimidos lib prolongad...,3,8054083018706.0
4,0.894066,124959.0,59751,rinvoq 30 miligramos 30 miligramos 30 comprimi...,rinvoq 30 miligramos comprimidos lib prolongad...,4,8054083021386.0
6,0.876686,124898.0,59720,skyrizi 150 miligramos 1 lapicera prellenada p...,skyrizi 150 miligramos lapicera prellenada 1,6,8054083021935.0
10,0.852467,119575.0,53119,venclexta kit inicio kit de inicio 1 pack 4 7 ...,venclexta kit de inicio 1,10,8054083013343.0
11,0.943270,119573.0,53120,venclexta 100 miligramos 100 miligramos 120 co...,venclexta 100 miligramos comprimidos 120,11,8054083013336.0
14,0.812576,119561.0,14916,sevorane 250 mililitros,sevorane envase 250 mililitros,14,8054083015156.0
17,1.000000,119577.0,44168,zemplar 2 microgramo 30 capsulas,zemplar 2 microgramo capsulas 30,17,8054083012032.0


INDEX DE SUSTRACCION PARA EL DF PROVEEDOR

In [174]:
#Sutraccion de productos sobre df ORIGINAL DE PROVEEDORES.
similiridad = 'similaridad'

umbral_similaridad = 0.9  # Ajusta este valor según tu caso
df_filtrado_similaridad = df_resultado_similiridad[df_resultado_similiridad[similiridad] > umbral_similaridad]

# Ahora, con los índices originales, extraemos los productos del df_Proveedor
# Filtramos por 'indice_original_proveedor' para obtener solo las filas que superaron el umbral
df_Proveedor_Similiridad_filtrado = df_Proveedor[df_Proveedor[variables_data.index_proveedor].isin(df_filtrado_similaridad['index_proveedor_x'])]

# Mostrar los productos filtrados del proveedor
df_Proveedor_Similiridad_filtrado


,condición de precios,cod. producto,descripcion,presentacion,unidades,precio s/iva,descuento,total,index_proveedor,descripcion_limpia_proveedor
2,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,2,humira ac 1 80 miligramos 1 lapicera prellenad...
11,NaN,20019067,VENCLEXTA 100mg,100 MG X 120 COMP.,NaN,NaN,NaN,NaN,11,venclexta 100 miligramos 100 miligramos 120 co...
17,NaN,20059449,ZEMPLAR 2 MCG,30 CAPSULAS,NaN,NaN,NaN,-,17,zemplar 2 microgramo 30 capsulas


In [175]:
df_resultado_similiridad.columns

#Arquitectura df_Productos

# *copia exacta de las columnas df_proveedor 
# xx_Niprod = Sistema Carena


Index(['similaridad', 'condición de precios_x', 'cod. producto_x',
       'descripcion_x', 'presentacion_x', 'unidades _x', 'precio s/iva_x',
       'descuento_x', 'total_x', 'index_proveedor_x',
       'descripcion_limpia_proveedor', 'Unnamed: 0', 'niprod', 'cod_prod',
       'cod_abrev', 'descrip1', 'descrip2', 'abrev', 'cod_catprod', 'codbarra',
       'cod_fbarra', 'unid_v', 'base_v', 'precio', 'mon_v', 'costo', 'base_c',
       'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c', 'unid_alt', 'rel_c_a',
       'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif', 'pped', 'st_min',
       'st_max', 'observ', 'pc_comision', 'pc_utilidad', 'ind_stock',
       'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto', 'ninum',
       'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item', 'cant_rep',
       'pto_rep', 'atrib0', 'atrib1', 'atrib2', 'index_producto',
       'descripcion_limpia_producto', 'condición de precios_y',
       'cod. producto_y', 'descripcion_y', 'presentacion_y', 'unidades _y',


MODELO DE PROCESAMIENTOS DE LENGUAJE NATURAL

Bert

In [176]:

modelo_nombre_BERT = 'dccuchile/bert-base-spanish-wwm-cased'

resultados_comparaciones = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_BERT,                # modelo_nombre
    "BERT",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Windows\Desktop\digitalizacion_distriapp\backend\src\data_utils\utilidades_comunes\aplicaciones_de_inteligencia_artificial\procesamiento_de_lenguaje_natural\index.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_master['numeros'] = df_master[columna_master].apply(extraer_numeros)  # Extrae números del master


KeyboardInterrupt: 

In [152]:
resultados_comparaciones.columns


Index(['descripcion_proveedor', 'descripcion_match', 'similitud',
       'index_proveedor', 'index_producto', 'niprod'],
      dtype='object')

In [153]:
# Unir df_Proveedor con resultados_comparaciones en base a index_proveedor
df_resultado = df_Proveedor.merge(resultados_comparaciones, 
                                  left_index=True,  # Índice de df_Proveedor (index_proveedor debe coincidir con el índice)
                                  right_on='index_proveedor',  # Columna en resultados_comparaciones
                                  how='right')  # Solo los que están en resultados_comparaciones

# Reorganizar las columnas si es necesario
df_resultado = df_resultado[df_Proveedor.columns.tolist() + ['niprod', 'similitud']]

# Mostrar el resultado
df_resultado

,condición de precios,cod. producto,descripcion,presentacion,unidades,precio s/iva,descuento,total,index_proveedor,descripcion_limpia_proveedor,numeros,niprod,similitud
0,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellenad...,"[2, 20, 2, 02]",119551.0,0.901277
1,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellenad...,"[2, 20, 2, 02]",119549.0,0.821588
2,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellenad...,"[2, 20, 2, 02]",119565.0,0.801296
3,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,humira ac 2 40 miligramos 2 lapiceras prellena...,"[2, 40, 2, 04]",119552.0,0.862091
4,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,humira ac 2 40 miligramos 2 lapiceras prellena...,"[2, 40, 2, 04]",119554.0,0.811453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NaN,93251AR,BOTOX,Vial 200 U,NaN,NaN,NaN,-,19,boto vial 200 unidad,[200],119577.0,0.730224
59,NaN,93251AR,BOTOX,Vial 200 U,NaN,NaN,NaN,-,19,boto vial 200 unidad,[200],119567.0,0.721654
60,NaN,93348AR,OZURDEX,0.7 MG C/1,NaN,NaN,NaN,-,20,ozurde 07 miligramos c 1,"[07, 1]",130998.0,0.851499
61,NaN,93348AR,OZURDEX,0.7 MG C/1,NaN,NaN,NaN,-,20,ozurde 07 miligramos c 1,"[07, 1]",119568.0,0.838913


In [ ]:
# Unir df_resultado con masterProductos en base a la columna niprod
df_resultado = df_resultado.merge(df_MasterProductos_Filtro[['niprod', variables_data.descripcion_limpia_MasterProductos]], 
                                  on='niprod',  # Clave de unión
                                  how='left').copy()  # 'left' para no perder filas en df_resultado

# Mostrar el resultado con la nueva columna agregada
#df_resultado[['niprod',"cod. producto",'presentacion','index_producto','similitud','index_proveedor']]


In [185]:
df_MasterProductos['xx_codigo_producto'] = None 
df_MasterProductos.columns

#Modelo del Nuevo csv del proveedor  para agregar a la master producto, si no se supera el umbral.
 
# niprod            = niprod  --> esto lo crea el sistema carena  por lo que no lo teng que crear
# descripcion       = descrip1
# presentacion      = descrip2	
# laboratorio       = atrib0
# codigo_producto   = codigo_producto  -> columna que se debe crear en la  MasterProducto



Index(['Unnamed: 0', 'niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2',
       'abrev', 'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v',
       'precio', 'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v',
       'rel_v_c', 'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado',
       'cod_bonif', 'pped', 'st_min', 'st_max', 'observ', 'pc_comision',
       'pc_utilidad', 'ind_stock', 'ind_partidas', 'ind_camestado',
       'ind_series', 'ind_fvto', 'ninum', 'tipo_egreso', 'pc_diftrans',
       'cod_cvta', 'cat_item', 'cant_rep', 'pto_rep', 'atrib0', 'atrib1',
       'atrib2', 'index_producto', 'descripcion_limpia_producto',
       'xx_codigo_producto'],
      dtype='object')

In [ ]:
df_resultado.columns



,condición de precios,cod. producto,descripcion,presentacion,unidades,precio s/iva,descuento,total,index_proveedor,descripcion_limpia_proveedor,numeros,niprod,similitud,descripcion_limpia_producto
0,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellenad...,"[2, 20, 2, 02]",119551.0,0.901277,humira ac 2 20 miligramos 20 miligramos 02 mil...
1,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellenad...,"[2, 20, 2, 02]",119549.0,0.821588,humira jeringa prellenada 2 08 mililitros
2,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellenad...,"[2, 20, 2, 02]",119565.0,0.801296,skyrizi 75 miligramos jeringa prellenada 2
3,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,humira ac 2 40 miligramos 2 lapiceras prellena...,"[2, 40, 2, 04]",119552.0,0.862091,humira ac 2 40 miligramos 40 miligramos 04 mil...
4,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,humira ac 2 40 miligramos 2 lapiceras prellena...,"[2, 40, 2, 04]",119554.0,0.811453,humira pen lapicera 2 08 mililitros
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NaN,93251AR,BOTOX,Vial 200 U,NaN,NaN,NaN,-,19,boto vial 200 unidad,[200],119577.0,0.730224,zemplar 2 microgramo capsulas 30
59,NaN,93251AR,BOTOX,Vial 200 U,NaN,NaN,NaN,-,19,boto vial 200 unidad,[200],119567.0,0.721654,survanta frasco ampolla 8 mililitros
60,NaN,93348AR,OZURDEX,0.7 MG C/1,NaN,NaN,NaN,-,20,ozurde 07 miligramos c 1,"[07, 1]",130998.0,0.851499,skyrizi 600 miligramos vial 1
61,NaN,93348AR,OZURDEX,0.7 MG C/1,NaN,NaN,NaN,-,20,ozurde 07 miligramos c 1,"[07, 1]",119568.0,0.838913,synagis 100 miligramos lioffa 1


In [190]:
df_temp = df_resultado[['niprod', 'descripcion', 'presentacion', 'cod. producto']].copy()

# Renombrar columnas
df_temp.rename(columns={
    'descripcion': 'descrip1',
    'presentacion': 'descrip2',
    'cod. producto': 'xx_codigo_producto'
}, inplace=True)

df_temp.columns

Index(['niprod', 'descrip1', 'descrip2', 'xx_codigo_producto'], dtype='object')

In [ ]:
# Unir ambos DataFrames sin filtrar por similitud
df_MasterProductos = pd.concat([df_MasterProductos, df_temp], ignore_index=True)
df_MasterProductos[['descrip1', 'descrip2']].iloc[57176:57185]

,descrip1,descrip2
57176,MAVIRET,100 MG/40 MG x 84 COMP REC
57177,MAVIRET,100 MG/40 MG x 84 COMP REC
57178,VENCLEXTA KIT INICIO,KIT DE INICIO X 1 - PACK 4X7 DIAS
57179,VENCLEXTA KIT INICIO,KIT DE INICIO X 1 - PACK 4X7 DIAS
57180,VENCLEXTA KIT INICIO,KIT DE INICIO X 1 - PACK 4X7 DIAS
57181,VENCLEXTA 100mg,100 MG X 120 COMP.
57182,VENCLEXTA 100mg,100 MG X 120 COMP.
57183,VENCLEXTA 100mg,100 MG X 120 COMP.
57184,VENCLEXTA 10mg,"10MG WALLET, TWO 10MG TABS X14"


PubMedBERT

In [157]:
"""
modelo_nombre_PubMedBert = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'

resultados_PubMedBERT = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_PubMedBert,                # modelo_nombre
    "PubMedBert",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)
"""

'\nmodelo_nombre_PubMedBert = \'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract\'\n\nresultados_PubMedBERT = procesar_comparacion(\n    df_MasterProductos_Filtro, \n    df_Proveedor, \n    modelo_nombre_PubMedBert,                # modelo_nombre\n    "PubMedBert",                # nombre_modelo\n    \'descripcion_limpia_producto\',  # columna_master\n    \'descripcion_limpia_proveedor\'  # columna_proveedor\n)\n'

BioBert

In [158]:
"""
modelo_nombre_BioBert = 'dmis-lab/biobert-base-cased-v1.1'

resultados_BioBert = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_BioBert,                # modelo_nombre
    "BioBert",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)
 """

'\nmodelo_nombre_BioBert = \'dmis-lab/biobert-base-cased-v1.1\'\n\nresultados_BioBert = procesar_comparacion(\n    df_MasterProductos_Filtro, \n    df_Proveedor, \n    modelo_nombre_BioBert,                # modelo_nombre\n    "BioBert",                # nombre_modelo\n    \'descripcion_limpia_producto\',  # columna_master\n    \'descripcion_limpia_proveedor\'  # columna_proveedor\n)\n '

FINE TUNING